In [7]:
INSTALL_PACKAGES = False
if INSTALL_PACKAGES:
    ! pip install --user rightmove-webscraper --quiet
    ! pip install pytesseract --quiet
    ! pip install requests --quiet
    ! pip install bs4 --quiet
    ! pip install pywebcopy --quiet
    ! pip install pycurl --quiet
    ! pip install selenium
    ! pip install keyboard
    ! pip install nltk

# Imports

In [210]:
from itertools import chain

In [194]:
from PIL import Image
from pytesseract import pytesseract
import pytesseract
import nltk

In [10]:
import pywebcopy
from rightmove_webscraper import RightmoveData
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [332]:
import os
import shutil
import re
import requests
from bs4 import BeautifulSoup
import json
import ukpropsearch_utils as utils
import importlib
importlib.reload(utils)

<module 'ukpropsearch_utils' from 'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\ukpropsearch_utils.py'>

## Utils

In [12]:
utils.only_text_containing(["gna-1", "gna-2", "zurli-1", "peppo-1"], ["gna", "zurli"])

['gna-2', 'zurli-1', 'gna-1']

## Rightmove string building

In [13]:
london_r3miles_url = "https://www.rightmove.co.uk/property-for-sale/find.html?minBedrooms=2&keywords=&dontShow=sharedOwnership%2Cretirement&channel=BUY&index=0&retirement=false&maxBedrooms=2&includeSSTC=false&partBuyPartRent=false&sortType=2&minPrice=400000&viewType=LIST&maxPrice=450000&radius=3.0&maxDaysSinceAdded=14&locationIdentifier=REGION%5E87490"

In [185]:
london_r3miles = RightmoveData(london_r3miles_url)
london_r3miles_results = london_r3miles.get_results
london_r3miles.summary()

,number_bedrooms,count,mean
0,2,552,449094.202899


In [453]:
test_property = london_r3miles_results.iloc[3]
test_property_url = test_property.url
test_property_url = test_property_url.split('#')[0] + "#/"
test_property_floorplan_url = test_property_url + "floorplan"
test_property_hashkey = re.sub(r'[^\w]', '', f"{test_property.address}")
print(test_property_hashkey)
print(test_property_url)
print(test_property_floorplan_url)

13TrobridgeParadeGrahameParkWayLondonNW9
http://www.rightmove.co.uk/properties/120961427#/
http://www.rightmove.co.uk/properties/120961427#/floorplan


In [454]:
html_data = requests.get(test_property_url).text

In [455]:
soup = BeautifulSoup(html_data, 'html.parser')
all_items = soup.find_all()
all_items_str = [str(item) for item in all_items]

In [456]:
with open('test.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(all_items_str))

In [457]:
img_urls_hd = utils.only_text_containing(all_items_str, [".png", ".jpg", ".jpeg"])
floorplan_imgs_hd = utils.only_text_containing(img_urls_hd, ["floorplan"], True)
floorplan_imgs_hd = utils.only_text_containing(floorplan_imgs_hd, ["FLP"])

cache_folder = f'{os.getcwd()}\\cache'
images_folder = f'{os.getcwd()}\\images'
floorplan_imgs_hd_download_folder = f"{cache_folder}\\{test_property_hashkey}"
floorplan_imgs_hd_download_fullpath = f"{floorplan_imgs_hd_download_folder}\\{test_property_hashkey}.html"

In [ ]:
utils.save_list_to_file(floorplan_imgs_hd)

In [ ]:
img_urls_hd = len(img_urls_hd)

In [ ]:
soup = BeautifulSoup(html_data, 'html.parser')
all_img_urls = soup.find_all('img')
for item in all_img_urls:
    if ("floorplan" in item["alt"].lower()):
        print(item["src"])

In [ ]:
kwargs = {'bypass_robots': True, 'project_name': test_property_hashkey}
# pywebcopy.save_webpage(test_property_url, download_folder, **kwargs) # this freezes after downloading a few files

In [461]:
import time
from selenium import webdriver
import keyboard

In [462]:
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get(test_property_floorplan_url);

In [445]:
if not os.path.exists(floorplan_imgs_hd_download_folder):
    os.mkdir(floorplan_imgs_hd_download_folder)

In [446]:
keyboard.press_and_release('ctrl+s')
time.sleep(0.5)
keyboard.write(floorplan_imgs_hd_download_fullpath)
keyboard.press_and_release('enter')

In [447]:
importlib.reload(utils)
image_paths = utils.find_files(['*.jpeg', "*.png", "*.jpg"], floorplan_imgs_hd_download_folder)
floorplan_img_hd_path = [image_path for image_path in image_paths if "FLP" in image_path][0]

IndexError: list index out of range

In [ ]:
# Copy the image in the 'images' folder
if not os.path.exists(images_folder):
    os.mkdir(images_folder)

floorplan_img_hd_filename = floorplan_img_hd_path.split("\\")[-1]

new_path = os.path.join(images_folder, floorplan_img_hd_filename)
shutil.copy2(floorplan_img_hd_path, new_path)
floorplan_img_hd_path = new_path

In [ ]:
# Clean the cache from this download
shutil.rmtree(floorplan_imgs_hd_download_folder)

# Text recognition

In [ ]:
floorplan_img_hd_path

In [ ]:
tesseract_install_path = r"C:\Program Files\Tesseract-OCR\tesseract.exe" # needs updating depending on x64-x86 and OS
pytesseract.pytesseract.tesseract_cmd = tesseract_install_path

In [ ]:
# Simple image to string
floorplan_text = pytesseract.image_to_string(Image.open(floorplan_img_hd_path)).split("\n")

In [ ]:
floorplan_text = [t.replace("\n", "") for t in floorplan_text if t]

In [ ]:
floorplan_text

In [ ]:
from nltk.corpus import wordnet
nltk.download("wordnet")

In [ ]:
room_hyponyms = wordnet.synsets('room')[0].hyponyms()

In [ ]:
room_hyponyms = [hyp.lemma_names() for hyp in room_hyponyms]

In [ ]:
# selectMany
room_names_set = set([hpy.replace('_', ' ') for hyp_lemma in room_hyponyms for hpy in hyp_lemma])

In [ ]:
# Outdoor space hyponyms
outdoor_space_hyponyms = ["patio", "garden", "terrace", "balcony"]
outdoor_space_hyponyms_communal = [f"communal {hyp}" for hyp in outdoor_space_hyponyms]
outdoor_space_hyponyms_private = [f"private {hyp}" for hyp in outdoor_space_hyponyms]

In [ ]:
# Add more names
room_names_set.update(["studio", "garden studio"])
room_names_set.update(outdoor_space_hyponyms)
room_names = list(room_names_set)

In [ ]:
# find room names in the floorplan
room_indexes = [floorplan_text.index(t) for t in floorplan_text if t.lower() in room_names]
rooms_found = [floorplan_text[index] for index in room_indexes]

In [ ]:
rooms_found

In [ ]:
# Floor number
# Generate all ordinal names for floor
floor_ordinal_names = ["basement", "lower ground", "ground"]
floor_ordinal_names.extend(["roof"])
floor_ordinal_names.extend(utils.ordinal(i) for i in range(1,100))

In [ ]:
internal_area_names = ["gross internal floor area", "floor area", "internal floor area"]
internal_area_names_text = list(set([text for text in floorplan_text for ian in internal_area_names if ian in text.lower()]))[0]

In [ ]:
area_unit_names = ["sqm", "sqmt", "sqft", "sqf"]

In [ ]:
internal_area_names_text

In [ ]:
internal_area_names[1] in floorplan_text[1]

In [ ]:
area_unit_names

In [ ]:
regex = re.compile(rf'[0-9\.\s]+(?:{"|".join(area_unit_names)})')
area_texts = regex.findall(internal_area_names_text.lower())
area_texts = [at.replace(" ","") for at in area_texts]
area_texts

In [ ]:
areas = [re.findall(r'[A-Za-z]+|\d+(?:\.\d+)?', s) for s in area_texts]
areas